## Riffle Shuffles:
A riffle shuffle is executed as follows: a deck of cards is split into two equal halves,<br>
with the top half taken in the left hand and the bottom half taken in the right hand.<br>
Next, the cards are interleaved exactly, with the top card in the right half inserted just<br>
after the top card in the left half, the 2nd card in the right half just after the 2nd card<br>
in the left half, etc.<p>
*Note that this process preserves the location of the top and bottom card of the deck<p>

Let s(n) be the minimum number of consecutive riffle shuffles needed to restore a deck<br>
of size n to its original configuration, where n is a positive even number.<p>

Amazingly, a standard deck of 52 cards will first return to its original configuration<br>
after only 8 perfect shuffles, so s(52)=8. It can be verified that a deck of 86 cards will<br>
also return to its original configuration after exactly 8 shuffles, and the sum of all values<br>
of n that satisfy s(n)=8 is 412.<p>

Find the sum of all values of n that satisfy s(n)=60.

### Solution:

#### A First Approach
First, I wrote out the rotations for a few of small deck sizes. Each time, I followed only<br>
the second card in the deck. It became clear that after each shuffle the 2 card had displaced<br>
by a powers of 2. Since the first card remains fixed under riffle shuffling, getting back to the<br>
original configuration requires $2^{n} - 1$ shuffles. Next, I wrote a function to test this conjecture for<br>
decks which return in exactly 8 steps. This first approach, while naive, turns out to be useful<br>
when reasoning further about the algorithm.

In [4]:
import Data.List

inExactly8 = f 2
  where
    f n | n > 2^8 = []
        | riffleId n == 8 = n : f (n+2)
        | otherwise = f (n+2)

riffleId n = f (riffleOnce [1..n]) [1..n] 1
  where
    half as = div (length as) 2
    takeHalf as = [take (half as) as, drop (half as) as]
    riffleOnce = concat . transpose . takeHalf
    f j const k | j == const = k
                | otherwise = f (riffleOnce j) const (k+1)

inExactly8

[18,52,86,256]

#### A More Refined Approach
While the above method is percise, explicitly calculating all riffle shuffles like a champ,<br>
this method does not scale very well and soon it becomes clear that to get a handle on<br>
deck sizes on the order of $2^{60}$, another method will be necessary.<p>

Noticing that every term in the list `inExactly8` is some $d+1$ where $d \mid 2^8-1$,<br>
another approach to the algorithm concerns itself with finding those divisors $d$ which<br>
divide $2^8-1$ but do not divide $2^k-1$ with $k$ a proper divisor of $d$. For example:<p>

$51 \mid 2^8-1$<br>
$51 \nmid 2^4-1$<br>
$51 \nmid 2^2-1$<br>

Further, any deck size larger than $2^n$ will not divide $2^n$ and so is a fine stopping point.<br>
With these points in mind, I arrived at this more refined algorithm:

In [19]:
refined622 = f 0
  where
    f n | n > 256 = []
        | dd n && cc n && bb n = n : f (n + 2)
        | otherwise = f (n + 2)

    bb n = mod 7 (n-1) /= 0
    cc n = mod 15 (n-1) /= 0
    dd n = mod 255 (n-1) == 0
    
refined622

[18,52,86,256]

Lo and behold, `refined622` lands the correct list. Moving forward with this approach<br>
in mind, I am ready to tackle the $60$ riffle shuffle case. $2^{60}-1$ is a rather large number,<br>
$1152921504606846975$, with very many divisors, $4608$. Being lazy, I don't wish to hard<br>
code these divisors and so, with the help of my favorite factorization library, I present a<br>
further refinement to the algorithm:

```
import Math.NumberTheory.Primes

main = do print.sum.anotherRefinement $ 60

anotherRefinement :: Integer -> [Integer]
anotherRefinement n = [ k + 1 | k <- listDivisors (2^n-1), divCond n k]
  where
    divCond n k = and [ mod (2^t-1) k /= 0 | t <- properDivisors n]

```

While the preceeding algorithm is spot on and not too slow, finishing<br>
in `(0.18 secs, 110,599,112 bytes)`. There certainly is a lot of double<br>
counting. This is a good place to find yet another refinement, perhaps<br>
one less beholden to iterated factoring.<p>

#### The insight: 13 binary dimensions and bit operators

Rather than querying every number between `1` and `1152921504606846975`, I can<br>
consider only those numbers which are factors of $2^{60}-1$. These factors can be<br>
generated directly by considering linear combinations of the prime factorization.<p>

`[(3,2),(5,2),(7,1),(11,1),(13,1),(31,1),(41,1),(61,1),(151,1),(331,1),(1321,1)]`<p>

Since almost every factor is unique, the list can nearly be interpreted as<br>
a basis for a vector space of factors. The repeated 3's and 5's do provide<br>
for a few hoops to jump through, but these problems are easily surmounted.<br>

#Todo: Reason through the bitwise operations#


In [10]:
import Data.Bits

euler622 = sum.remSort $ map ((+ 1).eval) goodBits

mapping = [3,3,5,5,7,11,13,31,41,151,331,1321,61]

goodBits :: [Integer]
goodBits = [n | n<-[1..2^13-1], bitCond n]
  where
    cond b = all (\c -> b .&. c /= b) 
    bitCond b | b <= 85   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90,89,87,86,85]
              | b <= 86   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90,89,87,86]
              | b <= 87   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90,89,87]
              | b <= 89   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90,89]
              | b <= 90   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91,90]
              | b <= 91   = cond b [1715,1714,1713,656,430,429,426,425,422,421,91]
              | b <= 421  = cond b [1715,1714,1713,656,430,429,426,425,422,421]
              | b <= 422  = cond b [1715,1714,1713,656,430,429,426,425,422]
              | b <= 425  = cond b [1715,1714,1713,656,430,429,426,425]
              | b <= 426  = cond b [1715,1714,1713,656,430,429,426]
              | b <= 429  = cond b [1715,1714,1713,656,430,429]
              | b <= 430  = cond b [1715,1714,1713,656,430]
              | b <= 656  = cond b [1715,1714,1713,656]
              | b <= 1713 = cond b [1715,1714,1713]
              | b <= 1714 = cond b [1715,1714]
              | b <= 1715 = cond b [1715]
              | otherwise = True

bitify [] = 0
bitify (n:ns) = n + bitify (map (* 2) ns)

listify 0 = [] 
listify n = mod n 2 :listify (div n 2)

eval bit = f (listify bit) mapping
  where
    f [] m  = 1
    f bs [] = 1
    f (b:bs) (m:ms) | b == 1 = m * (f bs ms)
                    | otherwise = f bs ms

-- dot product for lists
instance Num a => Num [a] where
  (*) as [] = []
  (*) [] as = []
  (*) (a:as) (b:bs) = a*b : as * bs

-- removes duplicates while sorting
remSort :: Ord a => [a] -> [a]
remSort [] = []
remSort [a] = [a]
remSort (a:as) = (remSort.smaller) (a:as) ++ [a] ++ (remSort.larger) (a:as)
  where
    smaller (x:xs) = filter (< x) xs
    larger (x:xs) = filter (> x) xs